### Preparación de datos

In [40]:
import pandas as pd
from pathlib import Path
import re
# === RUTAS ===

ruta_acta =  Path(r"20251003/20251003/ADP_DTM_FACT.Acta.csv")
ruta_insumo =  Path(r"20251003/20251003/ADP_DTM_DIM.Insumo.csv")
ruta_proyectos =  Path(r"20251003/20251003/ADP_DTM_DIM.Proyecto.csv")
ruta_capitulos = Path(r"20251003/20251003/ADP_DTM_DIM.CapituloPresupuesto.csv")
ruta_proyeccion =  Path(r"20251003/20251003/ADP_DTM_FACT.Proyeccion.csv")
ruta_items =  Path(r"20251003/20251003/ADP_DTM_DIM.Items.csv")
ruta_macro = Path(r"20251003/20251003/proyectos_macroproyectos.csv")
tabla_macro = pd.read_csv(ruta_macro)


In [41]:
# === CARGA ===
tabla_proyeccion = pd.read_csv(ruta_proyeccion)
tabla_items = pd.read_csv(ruta_items)
tabla_proyectos = pd.read_csv(ruta_proyectos)
tabla_capitulos = pd.read_csv(ruta_capitulos)
tabla_insumos = pd.read_csv(ruta_insumo)
tabla_macro = pd.read_csv(ruta_macro)

C:\Users\aleja\AppData\Local\Temp\ipykernel_33476\1549343039.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  tabla_items = pd.read_csv(ruta_items)


#### New Join

In [42]:
# Alinear columnas
if "Proyectos" in tabla_macro.columns:
    tabla_macro = tabla_macro.rename(columns={"Proyectos": "Nombre Proyecto"})
print("[DEBUG] Columnas alineadas.")


# Crear columna limpia sin prefijo en tabla_macro
tabla_macro["Proyecto_sin_prefijo"] = tabla_macro["Proyecto"].apply(
    lambda x: re.sub(r"^\s*\d+\s*-\s*", "", str(x)).strip() if pd.notna(x) else x
)

# Verificar coincidencias
interseccion = set(tabla_proyectos["Nombre Proyecto"]).intersection(set(tabla_macro["Proyecto"]))
print(f"[DEBUG] Coincidencias exactas entre tablas: {len(interseccion)}")

# === Merge de proyectos con macroproyectos ===
tabla_proyectos_macro = pd.merge(
    tabla_proyectos,
    tabla_macro,
    left_on="Nombre Proyecto",
    right_on="Proyecto",
    how="left",
)

print(f"[DEBUG] Merge proyectos + macroproyectos: {tabla_proyectos_macro.shape}")

# === PIPELINE COMPLETO ===
tabla_base = tabla_proyeccion.copy()

tabla_1 = pd.merge(tabla_base, tabla_proyectos_macro, on="SkIdProyecto", how="left")
tabla_2 = pd.merge(tabla_1, tabla_capitulos, on="SkIdCapitulo", how="left")

# Items
tabla_items_unica = tabla_items.drop_duplicates(subset=["SkIdItems"], keep="first")
tabla_3 = pd.merge(tabla_2, tabla_items_unica, on="SkIdItems", how="left", suffixes=("", "_item"))

# Insumos
tabla_insumos_unica = tabla_insumos.drop_duplicates(subset=["SkIdInsumo"], keep="first").copy()
cols_no_clave = [c for c in tabla_insumos_unica.columns if c != "SkIdInsumo"]
tabla_insumos_pref = tabla_insumos_unica.rename(columns={c: f"Insumo_{c}" for c in cols_no_clave})

tabla_final = pd.merge(tabla_3, tabla_insumos_pref, on="SkIdInsumo", how="left")
print(f"[DEBUG] OK. Forma final: {tabla_final.shape}")

# === Exportar resultados ===
tabla_final.to_csv("tabla_looker.csv", index=False, encoding="utf-8", sep=",")
tabla_final.to_parquet("tabla_looker.parquet", index=False)

print("[DEBUG] Exportado CSV y Parquet correctamente.")

# Vista rápida de verificación
display(tabla_final[["Nombre Proyecto", "Macroproyecto"]].head(10))

[DEBUG] Columnas alineadas.
[DEBUG] Coincidencias exactas entre tablas: 0
[DEBUG] Merge proyectos + macroproyectos: (85, 44)
[DEBUG] OK. Forma final: (273450, 114)
[DEBUG] Exportado CSV y Parquet correctamente.


,Nombre Proyecto,Macroproyecto
0,URBAN PLAZA,NaN
1,URBAN PLAZA,NaN
2,URBAN PLAZA,NaN
3,URBAN PLAZA,NaN
4,URBAN PLAZA,NaN
5,URBAN PLAZA,NaN
6,URBAN PLAZA,NaN
7,URBAN PLAZA,NaN
8,URBAN PLAZA,NaN
9,URBAN PLAZA,NaN


In [43]:
# === SELECCIÓN DE COLUMNAS ===
columnas_finales = [
    "SkIdProyecto", "SkIdCapitulo", "SkIdItems", "SkIdInsumo",
    "Nombre Proyecto", "Capitulo Descripcion", "Item Descripcion",
    "Insumo_Insumo Descripcion", "Insumo_Agrupacion Descripcion",
    "SkIdFecha Real", "Cantidad", "Valor Unitario", "Valor Total",
    "Insumo_Valor Unitario", "Insumo_Valor Neto", "Insumo_Fecha Creacion",
    "Cantidad Item", "Macroproyecto", "Insumo_Fecha Modificacion",
    "Fecha De Elaboracion", "Fecha De Inicio", "Fecha De Finalización",
    "SkIdFecha", "Capitulo Numero", "Cantidad_Item"
]

# Filtrar solo las columnas que existan realmente (por seguridad)
columnas_existentes = [col for col in columnas_finales if col in tabla_final.columns]
tabla_looker = tabla_final[columnas_existentes].copy()

print(f"[DEBUG] Columnas seleccionadas: {len(columnas_existentes)} / {len(columnas_finales)}")
print(f"[DEBUG] Forma final de tabla_looker: {tabla_looker.shape}")
display(tabla_looker.head(5))


[DEBUG] Columnas seleccionadas: 23 / 25
[DEBUG] Forma final de tabla_looker: (273450, 23)


,SkIdProyecto,SkIdCapitulo,SkIdItems,SkIdInsumo,Nombre Proyecto,Capitulo Descripcion,Item Descripcion,Insumo_Insumo Descripcion,Insumo_Agrupacion Descripcion,SkIdFecha Real,...,Insumo_Valor Unitario,Insumo_Valor Neto,Insumo_Fecha Creacion,Cantidad Item,Macroproyecto,Insumo_Fecha Modificacion,Fecha De Elaboracion,Fecha De Inicio,SkIdFecha,Capitulo Numero
0,1005,1005100,1006583,100279,URBAN PLAZA,CIMENTACION,Pilote preexcavado d=80cm,agua carrotanque,Campamento y provisionales,20110929,...,7327.59,8719.8321,06/12/2010,2.00,NaN,13/01/2011,28/03/2011,25/01/2011,20110929,2
1,1005,1005100,1006583,1002087,URBAN PLAZA,CIMENTACION,Pilote preexcavado d=80cm,concreto tremie 3000 psi grava común,Concretos Especiales,20110929,...,263115.84,313107.8496,06/12/2010,2.00,NaN,NaN,28/03/2011,25/01/2011,20110929,2
2,1005,1005100,1006583,1006063,URBAN PLAZA,CIMENTACION,Pilote preexcavado d=80cm,subcontrato pilote preexcavado d=0.80m,Subcontratos Preliminares Cimentación y Estruc...,20110929,...,40250.00,40250.0000,06/12/2010,2.00,NaN,27/05/2011,28/03/2011,25/01/2011,20110929,2
3,1005,1005100,1007643,100279,URBAN PLAZA,CIMENTACION,Pantalla preexcavada,agua carrotanque,Campamento y provisionales,20110930,...,7327.59,8719.8321,06/12/2010,32.28,NaN,13/01/2011,28/03/2011,25/01/2011,20110930,2
4,1005,1005100,1007643,1002087,URBAN PLAZA,CIMENTACION,Pantalla preexcavada,concreto tremie 3000 psi grava común,Concretos Especiales,20110930,...,263115.84,313107.8496,06/12/2010,32.28,NaN,NaN,28/03/2011,25/01/2011,20110930,2


#### Debug

In [30]:
# === MOSTRAR NOMBRES DE PROYECTOS Y MACROPROYECTOS ===

# Obtener listas únicas ordenadas
nombres_proyectos = sorted(tabla_proyectos["Nombre Proyecto"].dropna().unique().tolist())
nombres_macro = sorted(tabla_macro["Proyecto"].dropna().unique().tolist())

# Convertir a DataFrame comparativo
longitud_max = max(len(nombres_proyectos), len(nombres_macro))
tabla_comparativa = pd.DataFrame({
    "Nombre Proyecto (tabla_proyectos)": nombres_proyectos + [""] * (longitud_max - len(nombres_proyectos)),
    "Proyecto (proyectos_macroproyectos.csv)": nombres_macro + [""] * (longitud_max - len(nombres_macro))
})

print(f"[DEBUG] Proyectos en tabla_proyectos: {len(nombres_proyectos)}")
print(f"[DEBUG] Proyectos en proyectos_macroproyectos.csv: {len(nombres_macro)}")
print(f"[DEBUG] Mostrando tabla comparativa de nombres (orden alfabético)")

display(tabla_comparativa)


[DEBUG] Proyectos en tabla_proyectos: 85
[DEBUG] Proyectos en proyectos_macroproyectos.csv: 74
[DEBUG] Mostrando tabla comparativa de nombres (orden alfabético)


,Nombre Proyecto (tabla_proyectos),Proyecto (proyectos_macroproyectos.csv)
0,El Polo 1 - Etapa 1 - Torre 1,102 - Caminos de Sie - Urbanismo Externo MZ 2
1,ATRIO - Torre Norte,104 - Caminos de SIE - Edificaciones MZ2 - Et 1
2,BS Rosales,105 - Caminos de SIE - Edificaciones MZ2 - Et 2
3,CC Atrio - Circundantes Plaza Cívica,106 - Caminos de SIE - Edificaciones MZ2 - Et 3
4,CC Atrio - Plaza Cívica,107 - Caminos de SIE - Edificaciones MZ2 - Et 4
...,...,...
80,Valverde - Etapa Menta,
81,Valverde - Etapa Olivo,
82,Valverde - Palma,
83,Valverde - Roble,


#### quitar prefijo para ajustar nombres de macroproyectos

In [33]:
# === COMPARAR NOMBRES LIMPIOS DE MACROPROYECTOS VS tabla_proyectos ===
import pandas as pd
import re

def quitar_prefijo_numero(nombre: str) -> str:
    if pd.isna(nombre):
        return ""
    return re.sub(r"^\s*\d+\s*-\s*", "", str(nombre)).strip()

# Crear columna limpia en la tabla de macroproyectos
tabla_macro["Proyecto_sin_prefijo"] = tabla_macro["Proyecto"].apply(quitar_prefijo_numero)

# Obtener conjuntos únicos
set_proyectos = set(tabla_proyectos["Nombre Proyecto"].dropna().str.strip())
set_macro_limpio = set(tabla_macro["Proyecto_sin_prefijo"].dropna().str.strip())

# Calcular intersección y diferencias
interseccion = set_proyectos.intersection(set_macro_limpio)
faltantes_en_macro = set_proyectos - set_macro_limpio
faltantes_en_proyectos = set_macro_limpio - set_proyectos

print(f"[DEBUG] Coincidencias exactas (sin prefijo): {len(interseccion)}")
print(f"[DEBUG] Faltan en macroproyectos.csv (aun después de limpiar): {len(faltantes_en_macro)}")
print(f"[DEBUG] Faltan en tabla_proyectos: {len(faltantes_en_proyectos)}")

# Mostrar ejemplos de ambos
print("\n[DEBUG] Ejemplos de coincidencias:")
print(sorted(list(interseccion))[:5])

print("\n[DEBUG] Ejemplos de proyectos que faltan en macroproyectos:")
print(sorted(list(faltantes_en_macro))[:5])

print("\n[DEBUG] Ejemplos de proyectos que faltan en tabla_proyectos:")
print(sorted(list(faltantes_en_proyectos))[:5])


[DEBUG] Coincidencias exactas (sin prefijo): 67
[DEBUG] Faltan en macroproyectos.csv (aun después de limpiar): 18
[DEBUG] Faltan en tabla_proyectos: 7

[DEBUG] Ejemplos de coincidencias:
['BS Rosales', 'Caminos de SIE - Edificaciones MZ 4', 'Caminos de Sie - Edificaciones MZ1 - Et 1', 'Caminos de Sie - Edificaciones MZ1 - Et 2', 'Caminos de Sie - Edificaciones MZ1 - Et 3']

[DEBUG] Ejemplos de proyectos que faltan en macroproyectos:
['ATRIO - Torre Norte', 'CC Atrio - Circundantes Plaza Cívica', 'CC Atrio - Plaza Cívica', 'Caminos de Sie - Urbanismo Externo', 'Centro Cultural Atrio - Etapa 1']

[DEBUG] Ejemplos de proyectos que faltan en tabla_proyectos:
['Caminos de SIE - Edificaciones MZ2 - Et 1', 'Caminos de SIE - Edificaciones MZ2 - Et 2', 'Caminos de SIE - Edificaciones MZ2 - Et 3', 'Caminos de SIE - Edificaciones MZ2 - Et 4', 'Caminos de Sie - Urbanismo Externo MZ 2']


In [36]:
import re

if "Nombre Proyecto" in tabla_looker.columns and "Proyecto_sin_prefijo" not in tabla_looker.columns:
    tabla_looker["Proyecto_sin_prefijo"] = tabla_looker["Nombre Proyecto"].apply(
        lambda x: re.sub(r"^\s*\d+\s*-\s*", "", str(x)).strip() if pd.notna(x) else x
    )
    print("[DEBUG] Columna 'Proyecto_sin_prefijo' creada en tabla_looker.")

[DEBUG] Columna 'Proyecto_sin_prefijo' creada en tabla_looker.


# CONCATENAR DESCRIPCIONES JERÁRQUICAS Y MOSTRAR MUESTRA

In [44]:
# === CONCATENAR DESCRIPCIONES JERÁRQUICAS Y MOSTRAR MUESTRA ===

import pandas as pd

def construir_descripcion_jerarquica(fila: pd.Series,
                                     columnas_a_concatenar: list[str],
                                     separador: str = " - ") -> str:
    partes_limpias: list[str] = []
    for nombre_columna in columnas_a_concatenar:
        valor = fila.get(nombre_columna, "")
        if pd.isna(valor):
            continue
        valor_str = str(valor).strip()
        if valor_str:
            partes_limpias.append(valor_str)
    return separador.join(partes_limpias)

columnas_objetivo: list[str] = [
    "Macroproyecto",
    "Proyecto_sin_prefijo",
    "Capitulo Descripcion",
    "Item Descripcion",
    "Insumo_Insumo Descripcion",
]

columnas_que_existen = [c for c in columnas_objetivo if c in tabla_looker.columns]
columnas_que_faltan = [c for c in columnas_objetivo if c not in tabla_looker.columns]

print(f"[DEBUG] Columnas objetivo: {columnas_objetivo}")
print(f"[DEBUG] Columnas encontradas en 'tabla_looker': {columnas_que_existen}")
if columnas_que_faltan:
    print(f"[DEBUG] ADVERTENCIA: Faltan estas columnas y se omitirán: {columnas_que_faltan}")

nombre_columna_salida = "Descripcion_Jerarquica"
tabla_looker[nombre_columna_salida] = tabla_looker.apply(
    construir_descripcion_jerarquica,
    axis=1,
    columnas_a_concatenar=columnas_que_existen,
    separador=" - "
)




[DEBUG] Columnas objetivo: ['Macroproyecto', 'Proyecto_sin_prefijo', 'Capitulo Descripcion', 'Item Descripcion', 'Insumo_Insumo Descripcion']
[DEBUG] Columnas encontradas en 'tabla_looker': ['Macroproyecto', 'Capitulo Descripcion', 'Item Descripcion', 'Insumo_Insumo Descripcion']
[DEBUG] ADVERTENCIA: Faltan estas columnas y se omitirán: ['Proyecto_sin_prefijo']


In [45]:
# Mostrar solo filas con Macroproyecto definido
tamano_muestra: int = 100
muestra_pequena = (
    tabla_looker[
        tabla_looker["Macroproyecto"].notna()
        & (tabla_looker["Macroproyecto"].astype(str).str.strip() != "")
    ][[nombre_columna_salida]]
    .head(tamano_muestra)
)

print(f"[DEBUG] Tamaño de la tabla con la nueva columna: {tabla_looker.shape}")
print(f"[DEBUG] Mostrando las primeras {tamano_muestra} filas que tienen Macroproyecto definido:")
display(muestra_pequena)

[DEBUG] Tamaño de la tabla con la nueva columna: (273450, 24)
[DEBUG] Mostrando las primeras 100 filas que tienen Macroproyecto definido:


,Descripcion_Jerarquica


##### debug

In [28]:
# === DIAGNÓSTICO RÁPIDO DE MACROPROYECTOS VACÍOS ===

print("[DEBUG] Total de filas:", len(tabla_looker))
print("[DEBUG] Filas con Macroproyecto no nulo:",
      tabla_looker["Macroproyecto"].notna().sum())

# ¿Qué valores distintos hay en Macroproyecto?
print("[DEBUG] Valores únicos de Macroproyecto (primeros 10):")
print(tabla_looker["Macroproyecto"].dropna().unique()[:10])

# Si todos son NaN o '', veamos qué pasa con los nombres:
proyectos_en_tabla = set(tabla_looker["Nombre Proyecto"].dropna().unique())
ruta_macro = r"20251003/20251003/proyectos_macroproyectos.csv"
tabla_macro = pd.read_csv(ruta_macro)

# Detectar nombres coincidentes, sin normalizar
col_macro_nombre = "Proyecto" if "Proyecto" in tabla_macro.columns else "Proyectos"
proyectos_en_macro = set(tabla_macro[col_macro_nombre].dropna().unique())

interseccion = proyectos_en_tabla.intersection(proyectos_en_macro)
print(f"[DEBUG] Coincidencias exactas entre nombres: {len(interseccion)}")

if len(interseccion) == 0:
    print("[DEBUG] No hay coincidencias exactas de texto — probablemente difieren en mayúsculas, acentos o espacios.")


[DEBUG] Total de filas: 273450
[DEBUG] Filas con Macroproyecto no nulo: 0
[DEBUG] Valores únicos de Macroproyecto (primeros 10):
[]
[DEBUG] Coincidencias exactas entre nombres: 0
[DEBUG] No hay coincidencias exactas de texto — probablemente difieren en mayúsculas, acentos o espacios.


#### exportar

In [26]:
# === GUARDAR TABLA EN PARQUET ===
import os

# Ruta y nombre de salida (usa mismo nombre base que el CSV)
nombre_archivo_parquet = "tabla_looker.parquet"
ruta_salida_parquet = os.path.join(os.getcwd(), nombre_archivo_parquet)

# Guardar en formato Parquet
try:
    tabla_looker.to_parquet(ruta_salida_parquet, index=False)
    print(f"[DEBUG] Archivo guardado correctamente en formato Parquet: {ruta_salida_parquet}")
    print(f"[DEBUG] Tamaño de la tabla guardada: {tabla_looker.shape}")
except Exception as error:
    print(f"[DEBUG] ERROR al guardar el archivo Parquet: {error}")


[DEBUG] Archivo guardado correctamente en formato Parquet: c:\Users\aleja\Documents\Ingenieria Estadistica\Asignaturas2025B\arpro1\Base de Datos ARPRO\tabla_looker.parquet
[DEBUG] Tamaño de la tabla guardada: (273450, 24)
